# FIT file opening 
- fit file specification (zip download): https://www.thisisant.com/resources/fit

In [49]:
import os
import sys
import fitparse
import pandas as pd
sys.path.insert(0,'..')
from garmin_fit_eda.file_reader import messages_to_dataframe

In [50]:
DATA_ROOT = '../data'
INPUT_FILE = 'A5H81711.FIT'

## File opening using fitfile package

In [15]:
fitfile = fitparse.FitFile(os.path.join(DATA_ROOT, INPUT_FILE))

In [16]:
# Get list of messages
message_count = {}
for message in fitfile.get_messages():
    name = message.name
    if name not in message_count:
        message_count[name] = 1
    else:
        message_count[name] += 1

In [51]:
df = pd.DataFrame.from_dict(message_count, orient='index').reset_index()
df = df.rename(columns={'index': 'message_name', 0: 'counts'})
df = df.sort_values(by=['counts'], ascending=False)
df

,message_name,counts
14,hrv,4313
13,record,4022
15,unknown_104,14
17,unknown_216,13
3,device_info,12
16,lap,12
2,event,6
21,unknown_113,4
4,unknown_22,3
20,activity,1


## Get a different dataframe for each message group

In [55]:
df_dict = {}
for msg_type in message_count.keys():
    df_dict[msg_type] = messages_to_dataframe(fitfile.get_messages(msg_type), dump_units=True)

In [56]:
df_dict['event'].head()

,data,event,event_group,event_type,timer_trigger,timestamp
0,NaN,timer,0.0,start,manual,2020-05-17 06:17:11
1,100.0,39,NaN,marker,NaN,2020-05-17 06:23:10
2,NaN,timer,0.0,stop_all,manual,2020-05-17 07:24:12
3,123.0,recovery_hr,NaN,marker,NaN,2020-05-17 07:26:12
4,2746.0,38,NaN,marker,NaN,2020-05-17 07:30:47


In [57]:
df_dict['record'].head()

,activity_type,altitude,cadence,cadence_units,distance,distance_units,enhanced_altitude,enhanced_altitude_units,enhanced_speed,enhanced_speed_units,...,temperature,temperature_units,timestamp,unknown_87,unknown_88,unknown_90,vertical_oscillation,vertical_oscillation_units,vertical_ratio,vertical_ratio_units
0,walking,2616,52,rpm,2.23,m,23.2,m,1.484,m/s,...,29,C,2020-05-17 06:17:11,0,300,NaN,37.0,mm,4.96,percent
1,walking,2617,52,rpm,4.01,m,23.4,m,1.754,m/s,...,29,C,2020-05-17 06:17:12,0,300,NaN,41.0,mm,4.96,percent
2,walking,2621,52,rpm,6.08,m,24.2,m,1.754,m/s,...,29,C,2020-05-17 06:17:13,0,300,NaN,41.0,mm,7.40,percent
3,running,2616,78,rpm,8.58,m,23.2,m,1.829,m/s,...,29,C,2020-05-17 06:17:14,0,300,NaN,84.7,mm,12.00,percent
4,running,2615,79,rpm,11.06,m,23.0,m,2.099,m/s,...,29,C,2020-05-17 06:17:15,0,300,NaN,83.0,mm,11.50,percent


In [34]:
folder = os.path.split(INPUT_FILE)[1]
os.mkdir(folder)
for k, df in df_dict.items():
    df.to_html(os.path.join(folder, k+'.html'))

In [27]:
df_dict['event'].to_html('event.html')

In [23]:
df_dict['lap']

,avg_fractional_cadence,avg_heart_rate,avg_running_cadence,avg_speed,end_position_lat,end_position_long,enhanced_avg_speed,enhanced_max_speed,event,event_type,...,total_calories,total_descent,total_distance,total_elapsed_time,total_strides,total_timer_time,unknown_27,unknown_28,unknown_29,unknown_30
0,0.804688,148,86,2672,534810365,138767172,2.672,2.743,lap,stop,...,64,7,1000.00,374.306,540,374.306,534867487,138838990,534810365,138767172
1,0.375000,152,87,2811,534779635,138664311,2.811,3.284,lap,stop,...,64,46,1000.00,355.798,518,355.798,534810075,138766922,534767028,138664311
2,0.726562,151,88,2779,534862086,138584535,2.779,2.865,lap,stop,...,64,13,1000.00,359.793,531,359.793,534862086,138664043,534779876,138576618
3,0.500000,161,88,3135,534940763,138630048,3.135,3.284,lap,stop,...,61,60,1000.00,318.972,470,318.972,534942920,138630048,534862296,138583216
4,0.843750,165,87,3077,534881128,138754111,3.077,3.322,lap,stop,...,63,31,1000.00,324.985,475,324.985,534940591,138754111,534881128,138630558
5,0.890625,166,86,3007,534875693,138850166,3.007,3.126,lap,stop,...,65,29,1000.00,332.569,482,332.569,534880914,138850166,534853252,138754559
6,0.359375,166,85,3008,534874890,138853002,3.008,3.051,lap,stop,...,1,0,21.29,7.078,8,7.078,534875585,138853002,534874890,138850575


In [22]:
df_dict['unknown_104']

,unknown_0,unknown_2,unknown_253,unknown_3
0,3756,34,843246149,27
1,3752,33,843246449,27
2,3755,32,843246749,26
3,3751,30,843247049,26
4,3746,29,843247349,26
5,3738,28,843247649,25
6,3737,27,843247948,25


In [21]:
df_dict['unknown_22']

,unknown_0,unknown_1,unknown_2,unknown_253,unknown_3,unknown_4,unknown_5
0,1,1,3,843245920,1,2,3


In [20]:
df_dict['device_settings']

,active_time_zone,activity_tracker_enabled,autosync_min_steps,autosync_min_time,backlight_mode,mounting_side,move_alert_enabled,time_mode,time_offset,time_zone_offset,...,unknown_64,unknown_65,unknown_66,unknown_67,unknown_68,unknown_69,unknown_81,unknown_83,unknown_84,utc_offset
0,0,True,1000,60,manual,left,False,hour24,7200,0.0,...,0,1,2,2,0,3,1,0,0,0


In [19]:
df_dict['session']

,avg_fractional_cadence,avg_heart_rate,avg_running_cadence,avg_speed,enhanced_avg_speed,enhanced_max_speed,event,event_type,first_lap_index,max_fractional_cadence,...,total_calories,total_descent,total_distance,total_elapsed_time,total_strides,total_timer_time,total_training_effect,trigger,unknown_110,unknown_81
0,0.671875,157,87,2904,2.904,3.322,lap,stop,0,0.5,...,382,186,6021.29,2073.501,3024,2073.501,3.2,activity_end,Corsa,0


In [18]:
df_dict['file_creator']

,software_version
0,540


In [17]:
df_dict['activity']

,event,event_type,local_timestamp,num_sessions,timestamp,total_timer_time,type
0,activity,stop,2016-09-19 21:35:22,1,2016-09-19 19:35:22,2073.501,manual


In [15]:
df_dict['zones_target']

,hr_calc_type,max_heart_rate,unknown_254
0,custom,195,0


In [14]:
df_dict['device_info']

,ant_network,antplus_device_type,cum_operating_time,device_index,device_type,garmin_product,manufacturer,product,serial_number,software_version,source_type,timestamp,unknown_15,unknown_16,unknown_24,unknown_9
0,NaN,NaN,NaN,creator,NaN,fr230,garmin,NaN,3.913333e+09,5.4,local,2016-09-19 18:58:40,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1,0.0,1619,garmin,NaN,NaN,3.0,local,2016-09-19 18:58:40,NaN,NaN,NaN,NaN
2,antplus,heart_rate,2962298.0,2,NaN,NaN,NaN,2.0,NaN,3.0,antplus,2016-09-19 18:58:40,35238.0,1256.0,24665753.0,97.0
3,NaN,NaN,NaN,3,3.0,NaN,garmin_fr405_antfs,0.0,NaN,0.0,local,2016-09-19 18:58:40,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,creator,NaN,fr230,garmin,NaN,3.913333e+09,5.4,local,2016-09-19 19:33:13,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,1,0.0,1619,garmin,NaN,NaN,3.0,local,2016-09-19 19:33:13,NaN,NaN,NaN,NaN
6,antplus,heart_rate,2964362.0,2,NaN,NaN,NaN,2.0,NaN,3.0,antplus,2016-09-19 19:33:13,43428.0,1481.0,24665753.0,97.0
7,NaN,NaN,NaN,3,3.0,NaN,garmin_fr405_antfs,0.0,NaN,0.0,local,2016-09-19 19:33:13,NaN,NaN,NaN,NaN
